In [ ]:
# git status 에서 한글 꺠져 보일때
# git config --global core.quotepath false
# git config --global i18n.logOutputEncoding utf-8


from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFacePipeline
from typing import List, Literal, Optional, TypedDict
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langgraph.graph import StateGraph, END
import re
import pandas as pd

#emdeding_id = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
emdeding_id = "jhgan/ko-sbert-sts"

qa_model_id = "monologg/koelectra-base-v3-finetuned-korquad"
tokenizer_opt = {
    "max_length": 512, 
    "truncation": True,
}

# 1. 문서 로드 및 VectorDB 저장
filepath = "./NewBie_개발환경가이드.docx"
persist_directory = "./chromaDB_Chatbot"
#filepath = "./data/data.csv"
#persist_directory = "./ChromaDB_Chatbot_test"


# Word 문서 로드 함수
def load_word_documents(filepath: str):
    loader = UnstructuredWordDocumentLoader(filepath)
    docs = loader.load()
    all_text = "\n".join([doc.page_content for doc in docs])  # doc.page_content로 수정
    sentences = re.split(r'\.|\n', all_text)
    sentences = [s.strip() for s in sentences if s.strip()]
    print("문장 개수:", len(sentences))
    print("샘플 문장 10개:", sentences[:10])
    print("마지막 10문장:", sentences[-10:])
    return sentences


def load_csv_documents(filepath):
    df = pd.read_csv(filepath, encoding="utf-8")
    texts = df["text"].tolist()
    docs = [Document(page_content=text) for text in texts]
    return docs

# VectorDB 저장 함수
def save_vectorDB(docs, persist_directory, embedding_model_name):
    embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)
    vector_db = Chroma.from_documents(
        documents=docs,
        embedding=embedding_model,
        persist_directory=persist_directory
)
    return vector_db

# VectorDB에서 검색(retrieve) 함수
def load_vectorDB(persist_directory, embedding_model_name):
    embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)
    vector_db = Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding_model
)
    return vector_db




In [2]:
def run_qa(question, vector_db, qa_model_id, tokenizer_opt):
    retriever = vector_db.as_retriever(search_kwargs={"k":5})
    relevant_docs = retriever.invoke(question)
    if len(relevant_docs) == 0:
        print("검색 결과가 없습니다.")
        return None
    # 여러 문서를 context로 합침 (길이 증가)
    best_context = "\n".join([doc.page_content for doc in relevant_docs])
    print("[검색된 컨텍스트]", best_context)
    qa_pipeline = pipeline(
        task="question-answering",
        model=qa_model_id,
        tokenizer_kwargs=tokenizer_opt,
        device = -1
)
    result = qa_pipeline(
        question=question,
        context=best_context
)
    print("[QA 답변]", result['answer'])
    

In [3]:
#docs = load_csv_documents(filepath)
sentences = load_word_documents(filepath)
docs = [Document(page_content=s) for s in sentences]  # 각 문장을 Document로 변환
save_vectorDB(docs, persist_directory, emdeding_id)



문장 개수: 1905
샘플 문장 10개: ['Pull Request 과정', '1', 'Git action flow 의 이해', 'file Pre-merge (Essential 통합)', 'github/workflows/01_premerge_pull_request', 'yaml premerge test 주관하는 workflow a', 'Integration test', '/', 'github/workflows/shared_premerge_integration_test', 'yaml b']
마지막 10문장: ['커버리지 예시 (위의 workflow run중에 2번째)', '[AWIBOF-10103] add error id to log (#283) · poseidonos/pos-essential-orchestrator@2c322ac (github', 'com)', '클릭해서 가장 밑으로 스크롤하면 확인 가능', 'Trouble Shooting', '1', 'mockery 오류', '권한 문제', 'rsync등으로 코드만 미러링하여 해당 서버의 gopath에 필요한 패키지들이 없는 경우', 'go mod tidy 가 잘되면 문제없이 mockery 실행 가능']


In [4]:
vector_db = load_vectorDB(persist_directory, emdeding_id)
question = "TDK 는 무엇의 약자인가?"
run_qa(question, vector_db, qa_model_id, tokenizer_opt)

[검색된 컨텍스트] TDK는 다음과 같은 특징을 가지고 있습니다:
TDK는 이러한 문제를 해결하기 위해 만들어졌습니다
Test Development Kit (TDK)
TDK는 Test Development Kit의 약자로, 테스트를 빠르고 효과적으로 개발하기 위한 라이브러리 모음입니다
TDK는 테스트 개발을 빠르고 효과적으로 수행할 수 있도록 돕는 라이브러리 모음입니다


Device set to use cpu


[QA 답변] Test Development Kit의
